# Lesson 8 - Position Management Knowledge Graph Construction - Part I

With all the plans in place, it's time to construct the position management knowledge graph. 

For the **position domain graph** construction, no agent is required. The construction plan has all the information needed to drive a rule-based import of position hierarchy data.

<img src="images/domain.png" width="600">

**Note**: This notebook uses Cypher queries to build the position management domain graph from CSV files containing trades, positions, accounts, books, desks, legal entities, and instruments. Don't worry if you're unfamiliar with Cypher — focus on understanding the big picture of how the structured position data is transformed into a graph structure based on the construction plan.

## 8.1. Tool

A single tool which will build a position management knowledge graph using the defined construction rules.
- Input: `approved_construction_plan` (position hierarchy schema)
- Output: a position domain graph in Neo4j with trades, positions, accounts, books, desks, legal entities, and instruments
- Tools: `construct_domain_graph` + helper functions

**Workflow**

1. The context is initialized with an `approved_construction_plan` for position management and `approved_files`
2. Process all the node construction rules (Trade, Position, Account, Book, Desk, LegalEntity, Instrument, Counterparty)
3. Process all the relationship construction rules (BELONGS_TO, ALLOCATED_TO, REFERENCES, TRADES_WITH, etc.)

## 8.2. Setup

The usual import of needed libraries, loading of environment variables, and connection to Neo4j.

In [ ]:
# Import necessary libraries

from google.adk.models.lite_llm import LiteLlm # For OpenAI support

# Convenience libraries for working with Neo4j inside of Google ADK
from neo4j_for_adk import graphdb, tool_success, tool_error

from typing import Dict, Any

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.CRITICAL)

print("Libraries imported.")

In [ ]:
# --- Define Model Constants for easier use ---
MODEL_GPT_4O = "openai/gpt-4o"

llm = LiteLlm(model=MODEL_GPT_4O)

# Test LLM with a direct call
print(llm.llm_client.completion(model=llm.model, messages=[{"role": "user", "content": "Are you ready?"}], tools=[]))

print("\nOpenAI ready.")

In [ ]:
# Check connection to Neo4j by sending a query

neo4j_is_ready = graphdb.send_query("RETURN 'Neo4j is Ready!' as message")

print(neo4j_is_ready)

## 8.3. Tool Definitions (Domain Graph Construction)

The `construct_domain_graph` tool is responsible for constructing the "position management domain graph" from CSV files,
according to the approved construction plan for capital markets position hierarchy.

### Function: create_uniqueness_constraint



This function creates a uniqueness constraint in Neo4j to prevent duplicate nodes with the same label and property value from being created. This is critical for position management data to ensure unique trades, positions, accounts, and instruments.

In [ ]:
def create_uniqueness_constraint(
    label: str,
    unique_property_key: str,
) -> Dict[str, Any]:
    """Creates a uniqueness constraint for a node label and property key.
    A uniqueness constraint ensures that no two nodes with the same label and property key have the same value.
    This improves the performance and integrity of data import and later queries.

    Args:
        label: The label of the node to create a constraint for.
        unique_property_key: The property key that should have a unique value.

    Returns:
        A dictionary with a status key ('success' or 'error').
        On error, includes an 'error_message' key.
    """    
    # Use string formatting since Neo4j doesn't support parameterization of labels and property keys when creating a constraint
    constraint_name = f"{label}_{unique_property_key}_constraint"
    query = f"""CREATE CONSTRAINT `{constraint_name}` IF NOT EXISTS
    FOR (n:`{label}`)
    REQUIRE n.`{unique_property_key}` IS UNIQUE"""
    results = graphdb.send_query(query)
    return results


### Function: load_nodes_from_csv

This function performs batch loading of position management nodes from a CSV file into Neo4j. It uses the `LOAD CSV` command with the `MERGE` operation to create nodes (trades, positions, accounts, etc.) while avoiding duplicates based on the unique column (trade_id, position_id, account_id, etc.). The Cypher query processes data in batches of 1000 rows for better performance.

**Note**: The csv files are stored in the `/import` directory of `neo4j` database. When you use the query `LOAD CSV from "file:///" + $source_file`, neo4j checks the `/import` directory by default.

In [ ]:
def load_nodes_from_csv(
    source_file: str,
    label: str,
    unique_column_name: str,
    properties: list[str],
) -> Dict[str, Any]:
    """Batch loading of nodes from a CSV file"""

    # load nodes from CSV file by merging on the unique_column_name value
    query = f"""LOAD CSV WITH HEADERS FROM "file:///" + $source_file AS row
    CALL (row) {{
        MERGE (n:$($label) {{ {unique_column_name} : row[$unique_column_name] }})
        FOREACH (k IN $properties | SET n[k] = row[k])
    }} IN TRANSACTIONS OF 1000 ROWS
    """

    results = graphdb.send_query(query, {
        "source_file": source_file,
        "label": label,
        "unique_column_name": unique_column_name,
        "properties": properties
    })
    return results


### Execute Position Management Domain Graph Construction

This cell executes the main construction function using the approved construction plan. It builds the complete position management knowledge graph by importing all nodes (trades, positions, accounts, books, desks, legal entities, instruments) and relationships (position hierarchies, allocations, instrument references) according to the defined rules.

### Function: import_nodes

This function orchestrates the node import process for position management entities by first creating a uniqueness constraint (e.g., on trade_id, position_id, account_id) and then loading nodes from the CSV file. It ensures data integrity by establishing constraints before importing position data.

In [ ]:
def import_nodes(node_construction: dict) -> dict:
    """Import nodes as defined by a node construction rule."""

    # create a uniqueness constraint for the unique_column
    uniqueness_result = create_uniqueness_constraint(
        node_construction["label"],
        node_construction["unique_column_name"]
    )

    if (uniqueness_result["status"] == "error"):
        return uniqueness_result

    # import nodes from csv
    load_nodes_result = load_nodes_from_csv(
        node_construction["source_file"],
        node_construction["label"],
        node_construction["unique_column_name"],
        node_construction["properties"]
    )

    return load_nodes_result

### Function: import_relationships

This function imports relationships between position management nodes from a CSV file. It uses a Cypher query that matches existing nodes (trades, positions, accounts, books, desks, legal entities) and creates relationships between them (BELONGS_TO, ALLOCATED_TO, REFERENCES, etc.). The query finds pairs of nodes and creates relationships with specified properties (quantities, allocation percentages, timestamps) between them.

In [ ]:
def import_relationships(relationship_construction: dict) -> Dict[str, Any]:
    """Import relationships as defined by a relationship construction rule."""

    # load nodes from CSV file by merging on the unique_column_name value 
    from_node_column = relationship_construction["from_node_column"]
    to_node_column = relationship_construction["to_node_column"]
    query = f"""LOAD CSV WITH HEADERS FROM "file:///" + $source_file AS row
    CALL (row) {{
        MATCH (from_node:$($from_node_label) {{ {from_node_column} : row[$from_node_column] }}),
              (to_node:$($to_node_label) {{ {to_node_column} : row[$to_node_column] }} )
        MERGE (from_node)-[r:$($relationship_type)]->(to_node)
        FOREACH (k IN $properties | SET r[k] = row[k])
    }} IN TRANSACTIONS OF 1000 ROWS
    """
    
    results = graphdb.send_query(query, {
        "source_file": relationship_construction["source_file"],
        "from_node_label": relationship_construction["from_node_label"],
        "from_node_column": relationship_construction["from_node_column"],
        "to_node_label": relationship_construction["to_node_label"],
        "to_node_column": relationship_construction["to_node_column"],
        "relationship_type": relationship_construction["relationship_type"],
        "properties": relationship_construction["properties"]
    })
    return results

### Function: construct_domain_graph

This is the main orchestration function that builds the entire position management domain graph. It processes the construction plan in two phases:
1. **Node Construction**: First imports all position management nodes (Trade, Position, Account, Book, Desk, LegalEntity, Instrument, Counterparty) to ensure they exist before creating relationships
2. **Relationship Construction**: Then creates relationships between the existing nodes (BELONGS_TO for hierarchy, ALLOCATED_TO for allocations, REFERENCES for instruments, etc.)

This two-phase approach prevents relationship creation failures due to missing nodes and ensures proper position hierarchy construction.

In [ ]:
def construct_domain_graph(construction_plan: dict) -> Dict[str, Any]:
    """Construct a domain graph according to a construction plan."""
    # first, import nodes
    node_constructions = [value for value in construction_plan.values() if value['construction_type'] == 'node']
    for node_construction in node_constructions:
        import_nodes(node_construction)

    # second, import relationships
    relationship_constructions = [value for value in construction_plan.values() if value['construction_type'] == 'relationship']
    for relationship_construction in relationship_constructions:
        import_relationships(relationship_construction)

## 8.4. Run construct_domain_graph()

This cell defines the approved construction plan as a dictionary containing rules for creating position management nodes and relationships. The plan includes:

- **Node Rules**: Define how to create Trade, Position, Account, Book, Desk, LegalEntity, Instrument, and Counterparty nodes from CSV files
- **Relationship Rules**: Define how to create BELONGS_TO (position hierarchy), ALLOCATED_TO (trade allocations), REFERENCES (instrument links), and TRADES_WITH (counterparty) relationships

Each rule specifies the source file, labels, unique identifiers (trade_id, position_id, account_id, etc.), and properties (quantities, amounts, timestamps) to be imported.

In [ ]:
# the approved construction plan for position management should look something like this...
approved_construction_plan = {
    "Trade": {
        "construction_type": "node", 
        "source_file": "trades.csv", 
        "label": "Trade", 
        "unique_column_name": "trade_id", 
        "properties": ["trade_date", "trade_type", "quantity", "price", "amount", "currency", "trader_id", "book_id", "instrument_id", "counterparty_id"]
    }, 
    "Position": {
        "construction_type": "node", 
        "source_file": "positions.csv", 
        "label": "Position", 
        "unique_column_name": "position_id", 
        "properties": ["position_date", "quantity", "market_value", "currency", "account_id", "instrument_id"]
    }, 
    "Account": {
        "construction_type": "node", 
        "source_file": "accounts.csv", 
        "label": "Account", 
        "unique_column_name": "account_id", 
        "properties": ["account_number", "account_name", "account_type", "book_id", "opening_date"]
    }, 
    "Book": {
        "construction_type": "node", 
        "source_file": "books.csv", 
        "label": "Book", 
        "unique_column_name": "book_id", 
        "properties": ["book_code", "book_name", "desk_id", "region"]
    },
    "Desk": {
        "construction_type": "node", 
        "source_file": "desks.csv", 
        "label": "Desk", 
        "unique_column_name": "desk_id", 
        "properties": ["desk_name", "desk_type", "legal_entity_id", "location"]
    },
    "LegalEntity": {
        "construction_type": "node", 
        "source_file": "legal_entities.csv", 
        "label": "LegalEntity", 
        "unique_column_name": "entity_id", 
        "properties": ["entity_name", "lei_code", "jurisdiction", "regulatory_status"]
    },
    "Instrument": {
        "construction_type": "node", 
        "source_file": "instruments.csv", 
        "label": "Instrument", 
        "unique_column_name": "instrument_id", 
        "properties": ["isin", "cusip", "ticker", "instrument_name", "asset_class", "currency", "issuer"]
    },
    "Counterparty": {
        "construction_type": "node", 
        "source_file": "counterparties.csv", 
        "label": "Counterparty", 
        "unique_column_name": "counterparty_id", 
        "properties": ["counterparty_name", "lei_code", "counterparty_type", "rating", "jurisdiction"]
    },
    "BELONGS_TO_ACCOUNT": {
        "construction_type": "relationship", 
        "source_file": "positions.csv", 
        "relationship_type": "BELONGS_TO", 
        "from_node_label": "Position", 
        "from_node_column": "position_id", 
        "to_node_label": "Account", 
        "to_node_column": "account_id", 
        "properties": ["quantity"]
    }, 
    "BELONGS_TO_BOOK": {
        "construction_type": "relationship", 
        "source_file": "accounts.csv", 
        "relationship_type": "BELONGS_TO", 
        "from_node_label": "Account", 
        "from_node_column": "account_id", 
        "to_node_label": "Book", 
        "to_node_column": "book_id", 
        "properties": []
    },
    "BELONGS_TO_DESK": {
        "construction_type": "relationship", 
        "source_file": "books.csv", 
        "relationship_type": "BELONGS_TO", 
        "from_node_label": "Book", 
        "from_node_column": "book_id", 
        "to_node_label": "Desk", 
        "to_node_column": "desk_id", 
        "properties": []
    },
    "BELONGS_TO_ENTITY": {
        "construction_type": "relationship", 
        "source_file": "desks.csv", 
        "relationship_type": "BELONGS_TO", 
        "from_node_label": "Desk", 
        "from_node_column": "desk_id", 
        "to_node_label": "LegalEntity", 
        "to_node_column": "legal_entity_id", 
        "properties": []
    },
    "REFERENCES_INSTRUMENT_POSITION": {
        "construction_type": "relationship", 
        "source_file": "positions.csv", 
        "relationship_type": "REFERENCES", 
        "from_node_label": "Position", 
        "from_node_column": "position_id", 
        "to_node_label": "Instrument", 
        "to_node_column": "instrument_id", 
        "properties": []
    },
    "REFERENCES_INSTRUMENT_TRADE": {
        "construction_type": "relationship", 
        "source_file": "trades.csv", 
        "relationship_type": "REFERENCES", 
        "from_node_label": "Trade", 
        "from_node_column": "trade_id", 
        "to_node_label": "Instrument", 
        "to_node_column": "instrument_id", 
        "properties": []
    },
    "ALLOCATED_TO": {
        "construction_type": "relationship", 
        "source_file": "trade_allocations.csv", 
        "relationship_type": "ALLOCATED_TO", 
        "from_node_label": "Trade", 
        "from_node_column": "trade_id", 
        "to_node_label": "Account", 
        "to_node_column": "account_id", 
        "properties": ["allocation_quantity", "allocation_percentage", "allocation_timestamp"]
    },
    "TRADES_WITH": {
        "construction_type": "relationship", 
        "source_file": "trades.csv", 
        "relationship_type": "TRADES_WITH", 
        "from_node_label": "Trade", 
        "from_node_column": "trade_id", 
        "to_node_label": "Counterparty", 
        "to_node_column": "counterparty_id", 
        "properties": []
    }
}


In [ ]:
construct_domain_graph(approved_construction_plan)

## 8.5 Inspect the Position Management Domain Graph

This cell filters the construction plan to extract only the relationship construction rules. This list will be used in the next cell to verify that all relationships were successfully created in the graph.

In [ ]:
# extract a list of the relationship construction rules
relationship_constructions = [
    value for value in approved_construction_plan.values()
    if value.get("construction_type") == "relationship"
]
relationship_constructions

This cell creates and executes a Cypher query to verify that all relationship types from the position management construction plan were successfully created in the graph. 

The query uses several advanced Cypher features:
- `UNWIND`: Iterates through each relationship construction rule
- `CALL (construction) { ... }`: Subquery that executes for each construction rule
- `MATCH (from)-[r:relationship_type]->(to)`: Finds one example of each relationship type (BELONGS_TO, ALLOCATED_TO, REFERENCES, TRADES_WITH)
- `LIMIT 1`: Returns only one example per relationship type

This provides a summary view showing one instance of each relationship pattern in the constructed position management graph, verifying the complete hierarchy from Trade through Position, Account, Book, Desk to LegalEntity.

In [ ]:
# a fancy cypher query which to show one instance of each construction rule

# turn the list of rules into multiple single rules
unwind_list = "UNWIND $relationship_constructions AS construction"

# match a single path for a given construction.relationship_type
# return only the labels and types from the 3 parts of the path
match_one_path = """
    MATCH (from)-[r:$(construction.relationship_type)]->(to)
    RETURN labels(from) AS fromNode, type(r) AS relationship, labels(to) AS toNode
    LIMIT 1
"""
match_in_subquery = f"""
CALL (construction) {{
{match_one_path}
}}
"""

cypher = f"""
{unwind_list}
{match_in_subquery}
RETURN fromNode, relationship, toNode
"""

print(cypher)

print("\n---")

graphdb.send_query(cypher, {
    "relationship_constructions": relationship_constructions
})